<a href="https://colab.research.google.com/github/zareefrj/Final-Year-Project/blob/main/FYP_ARIMA_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ARIMA Forecasting**

**ARIMA Pipeline:**
1. Collect data from each place.
2. Check Stationarity
3. Train
4. Forecast

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
import plotly.graph_objs as go

Collect data from each site

In [ ]:

# Load the CSV file into a DataFrame
data = pd.read_csv('weatherAUS.csv')

# Assuming your CSV file has columns named 'City', 'Date', and 'Value'
cities = data['Location'].unique()

# Convert 'Date' column to datetime data type
data['Date'] = pd.to_datetime(data['Date'], dayfirst= True)

# Create a dictionary to store DataFrames for each city
city_data_frames = {}

# Iterate through each city and create a DataFrame for that city's data
for city in cities:
    city_data = data[data['Location'] == city]
    city_data_frames[city] = city_data[['Date', 'MaxTemp']].reset_index(drop=True)


In [ ]:
# Now you have separate DataFrames for each city in the city_data_frames dictionary
# You can access them using city names as keys, e.g., city_data_frames['City1'], city_data_frames['City2'], ...

# Example usage:
# To access the data for a specific city:
city1_data = city_data_frames['Melbourne']

print(city1_data)
latest_date = city1_data['Date'].max()
print(latest_date)

# Now you can feed these DataFrames to your ARIMA model for training

           Date  MaxTemp
0    2008-07-01     15.4
1    2008-07-02     14.5
2    2008-07-03     14.1
3    2008-07-04     14.5
4    2008-07-05     15.5
...         ...      ...
3188 2017-06-21     13.5
3189 2017-06-22     14.1
3190 2017-06-23     14.9
3191 2017-06-24     12.3
3192 2017-06-25     14.3

[3193 rows x 2 columns]
2017-06-25 00:00:00


Check Stationarity

In [ ]:

def adf_test(dataset): #Augmented Dickey–Fuller test
     dftest = adfuller(dataset, autolag = 'AIC') #Akaike information criterion
     """print("1. ADF : ",dftest[0])
     print("2. P-Value : ", dftest[1])
     print("3. Num Of Lags : ", dftest[2])
     print("4. Num Of Observations Used For ADF Regression:",      dftest[3])
     print("5. Critical Values :")
     for key, val in dftest[4].items():
         print("\t",key, ": ", val)
"""
     if dftest[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary")
     else:
        print("weak evidence against null hypothesis,indicating it is non-stationary ")
        dataset_diff = dataset - dataset.shift()
        dataset_diff = dataset_diff.dropna()
        print("Differencing Done")

for city, city_data in city_data_frames.items():
    column_name = 'MaxTemp'  # Replace with the name of the column you want to test
    print(f"ADF Test for {city} - {column_name}")
    adf_test(city_data[column_name].dropna())
    print("\n")

ADF Test for Canberra - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Sydney - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Melbourne - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Brisbane - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Adelaide - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Perth - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Hobart - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary


ADF Test for Darwin - MaxTemp
strong evidence against the null hypothesis(Ho), reject the null h

Training & Forecasting

In [ ]:
def fit_arima_and_predict(data, order=(3, 1, 8), forecast_steps=7):
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=forecast_steps)

    return forecast

# Perform ARIMA modeling and prediction for each city's 'MaxTemp'
for city, city_data in city_data_frames.items():
    column_name = 'MaxTemp'  # Replace with the name of the column you want to forecast
    print(f"Fitting ARIMA model and making predictions for {city} - {column_name}")

    # Get the latest date for the city
    latest_date = city_data['Date'].max()
    latest_data = city_data[city_data['Date'] == latest_date]

    # Fit ARIMA model
    order = (3, 1, 8)  # ARIMA order (p, d, q)
    forecast_steps = 7  # Number of days to forecast

    predictions = fit_arima_and_predict(latest_data[column_name], order=order, forecast_steps=forecast_steps)

    # Create a table to display predictions
    prediction_dates = pd.date_range(start=latest_date, periods=forecast_steps, freq='D')
    prediction_table = pd.DataFrame({'Date': prediction_dates, 'Predicted_MaxTemp': predictions})

    # Display the prediction table for the current city
    print(f"Predictions for {city} - {column_name}:\n")
    print(prediction_table)
    print("\n")

Fitting ARIMA model and making predictions for Canberra - MaxTemp
Predictions for Canberra - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               14.5
2 2017-06-26               14.5
3 2017-06-27               14.5
4 2017-06-28               14.5
5 2017-06-29               14.5
6 2017-06-30               14.5
7 2017-07-01               14.5


Fitting ARIMA model and making predictions for Sydney - MaxTemp
Predictions for Sydney - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               19.3
2 2017-06-26               19.3
3 2017-06-27               19.3
4 2017-06-28               19.3
5 2017-06-29               19.3
6 2017-06-30               19.3
7 2017-07-01               19.3


Fitting ARIMA model and making predictions for Melbourne - MaxTemp
Predictions for Melbourne - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               14.3
2 2017-06-26               14.3
3 2017-06-27               14.3
4 2017-06-28               14.3
5 2017-06-29     

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported inde

Predictions for Adelaide - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25                0.0
2 2017-06-26                0.0
3 2017-06-27                0.0
4 2017-06-28                0.0
5 2017-06-29                0.0
6 2017-06-30                0.0
7 2017-07-01                0.0


Fitting ARIMA model and making predictions for Perth - MaxTemp
Predictions for Perth - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               17.0
2 2017-06-26               17.0
3 2017-06-27               17.0
4 2017-06-28               17.0
5 2017-06-29               17.0
6 2017-06-30               17.0
7 2017-07-01               17.0


Fitting ARIMA model and making predictions for Hobart - MaxTemp
Predictions for Hobart - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               12.6
2 2017-06-26               12.6
3 2017-06-27               12.6
4 2017-06-28               12.6
5 2017-06-29               12.6
6 2017-06-30               12.6
7 2017-07-01               

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported inde

Predictions for Newcastle - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-24               19.9
2 2017-06-25               19.9
3 2017-06-26               19.9
4 2017-06-27               19.9
5 2017-06-28               19.9
6 2017-06-29               19.9
7 2017-06-30               19.9


Fitting ARIMA model and making predictions for Penrith - MaxTemp
Predictions for Penrith - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               20.0
2 2017-06-26               20.0
3 2017-06-27               20.0
4 2017-06-28               20.0
5 2017-06-29               20.0
6 2017-06-30               20.0
7 2017-07-01               20.0


Fitting ARIMA model and making predictions for Wollongong - MaxTemp
Predictions for Wollongong - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               18.1
2 2017-06-26               18.1
3 2017-06-27               18.1
4 2017-06-28               18.1
5 2017-06-29               18.1
6 2017-06-30               18.1
7 2017-07-01  

...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate start

Predictions for PerthAirport - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               16.8
2 2017-06-26               16.8
3 2017-06-27               16.8
4 2017-06-28               16.8
5 2017-06-29               16.8
6 2017-06-30               16.8
7 2017-07-01               16.8


Fitting ARIMA model and making predictions for SalmonGums - MaxTemp
Predictions for SalmonGums - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               12.6
2 2017-06-26               12.6
3 2017-06-27               12.6
4 2017-06-28               12.6
5 2017-06-29               12.6
6 2017-06-30               12.6
7 2017-07-01               12.6


Fitting ARIMA model and making predictions for Walpole - MaxTemp
Predictions for Walpole - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               15.4
2 2017-06-26               15.4
3 2017-06-27               15.4
4 2017-06-28               15.4
5 2017-06-29               15.4
6 2017-06-30               15.4
7 2017-07-0

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported inde

Predictions for Nhil - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25               14.3
2 2017-06-26               14.3
3 2017-06-27               14.3
4 2017-06-28               14.3
5 2017-06-29               14.3
6 2017-06-30               14.3
7 2017-07-01               14.3


Fitting ARIMA model and making predictions for Katherine - MaxTemp
Predictions for Katherine - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25                0.0
2 2017-06-26                0.0
3 2017-06-27                0.0
4 2017-06-28                0.0
5 2017-06-29                0.0
6 2017-06-30                0.0
7 2017-07-01                0.0


Fitting ARIMA model and making predictions for Uluru - MaxTemp
Predictions for Uluru - MaxTemp:

        Date  Predicted_MaxTemp
1 2017-06-25                0.0
2 2017-06-26                0.0
3 2017-06-27                0.0
4 2017-06-28                0.0
5 2017-06-29                0.0
6 2017-06-30                0.0
7 2017-07-01             

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: